In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_28_l1_eps_lmbd'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_28_l1_eps_lmbd-2024-11-28-19-15-57


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.025, "lambda": 0.0},
    {"epochs": 2, "epsilon": 0.01, "lambda": 0.1},
    {"epochs": 2, "epsilon": 0.0025, "lambda": 0.2},
    {"epochs": 4, "epsilon": 0.0, "lambda": 0.5}
]

In [6]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir


obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.train()
    infered_obj_path = result_dir / f"bunny-{current_date}-currciulum-{i}.obj"
    generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)
train_dataset.close()
val_dataset.close()

  0%|          | 0/37500 [00:00<?, ?it/s]

{'loss': 0.0028, 'grad_norm': 0.09028446674346924, 'learning_rate': 4.9986666666666674e-05, 'epoch': 0.0}
{'loss': 0.0035, 'grad_norm': 0.16022853553295135, 'learning_rate': 4.997333333333333e-05, 'epoch': 0.0}
{'loss': 0.0035, 'grad_norm': 0.03466399013996124, 'learning_rate': 4.996e-05, 'epoch': 0.0}
{'loss': 0.0036, 'grad_norm': 0.03947990760207176, 'learning_rate': 4.994666666666667e-05, 'epoch': 0.0}
{'loss': 0.003, 'grad_norm': 0.07916592806577682, 'learning_rate': 4.993333333333334e-05, 'epoch': 0.0}
{'loss': 0.0046, 'grad_norm': 0.12280762195587158, 'learning_rate': 4.992e-05, 'epoch': 0.0}
{'loss': 0.0037, 'grad_norm': 0.0782245472073555, 'learning_rate': 4.990666666666667e-05, 'epoch': 0.0}
{'loss': 0.0034, 'grad_norm': 0.013459913432598114, 'learning_rate': 4.989333333333334e-05, 'epoch': 0.0}
{'loss': 0.0032, 'grad_norm': 0.040632519870996475, 'learning_rate': 4.9880000000000004e-05, 'epoch': 0.0}
{'loss': 0.0032, 'grad_norm': 0.062187373638153076, 'learning_rate': 4.986666

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.00046344890142790973, 'eval_runtime': 149.3924, 'eval_samples_per_second': 1673.445, 'eval_steps_per_second': 41.836, 'epoch': 1.0}
{'loss': 0.0016, 'grad_norm': 0.06047657132148743, 'learning_rate': 2.4986666666666666e-05, 'epoch': 1.0}
{'loss': 0.0006, 'grad_norm': 0.0, 'learning_rate': 2.4973333333333334e-05, 'epoch': 1.0}
{'loss': 0.0003, 'grad_norm': 0.12255454808473587, 'learning_rate': 2.496e-05, 'epoch': 1.0}
{'loss': 0.0002, 'grad_norm': 0.0, 'learning_rate': 2.494666666666667e-05, 'epoch': 1.0}
{'loss': 0.0, 'grad_norm': 0.0, 'learning_rate': 2.4933333333333334e-05, 'epoch': 1.0}
{'loss': 0.001, 'grad_norm': 0.0, 'learning_rate': 2.4920000000000002e-05, 'epoch': 1.0}
{'loss': 0.0001, 'grad_norm': 0.08040677756071091, 'learning_rate': 2.4906666666666666e-05, 'epoch': 1.0}
{'loss': 0.0005, 'grad_norm': 0.0, 'learning_rate': 2.4893333333333334e-05, 'epoch': 1.0}
{'loss': 0.0002, 'grad_norm': 0.10273353010416031, 'learning_rate': 2.488e-05, 'epoch': 1.0}
{'loss': 

  0%|          | 0/6250 [00:00<?, ?it/s]

...

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0024835672229528427, 'eval_runtime': 171.0971, 'eval_samples_per_second': 1461.158, 'eval_steps_per_second': 36.529, 'epoch': 3.0}
{'loss': 0.0029, 'grad_norm': 0.14135098457336426, 'learning_rate': 1.2493333333333333e-05, 'epoch': 3.0}
{'loss': 0.0022, 'grad_norm': 0.5096641778945923, 'learning_rate': 1.2486666666666667e-05, 'epoch': 3.0}
{'loss': 0.0028, 'grad_norm': 0.3466844856739044, 'learning_rate': 1.248e-05, 'epoch': 3.0}
{'loss': 0.0018, 'grad_norm': 0.48361408710479736, 'learning_rate': 1.2473333333333335e-05, 'epoch': 3.0}
{'loss': 0.0022, 'grad_norm': 0.591457724571228, 'learning_rate': 1.2466666666666667e-05, 'epoch': 3.0}
{'loss': 0.0021, 'grad_norm': 0.43835994601249695, 'learning_rate': 1.2460000000000001e-05, 'epoch': 3.0}
{'loss': 0.0027, 'grad_norm': 0.5305781960487366, 'learning_rate': 1.2453333333333333e-05, 'epoch': 3.0}
{'loss': 0.002, 'grad_norm': 0.45200034976005554, 'learning_rate': 1.2446666666666667e-05, 'epoch': 3.0}
{'loss': 0.0032, 'grad_n

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.002295391634106636, 'eval_runtime': 169.2053, 'eval_samples_per_second': 1477.495, 'eval_steps_per_second': 36.937, 'epoch': 4.0}
{'train_runtime': 3940.368, 'train_samples_per_second': 761.35, 'train_steps_per_second': 19.034, 'train_loss': 0.0026348558790112537, 'epoch': 4.0}


Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]